# Learning basic of OpenVINO API through a simple image classification program
Here, you'll learn the basic of OpenVINO Python API through a simple image classification program.

### Preparing an input image and label text data files
First, let's prepare imput image file and class label text file. Those files are in the OpenVINO install directory. We'll simply copy them to the current working directory.

In [ ]:
# Linux
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car.png .
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/squeezenet1.1.labels synset_words.txt

In [ ]:
# Windows
!echo %INTEL_OPENVINO_DIR%
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\car.png" .
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\squeezenet1.1.labels" synset_words.txt

Show the image file and check the picture.

In [ ]:
from IPython.display import Image
Image('car.png')

### Preparing a DL model for inferencing
Download a DL model for image classification using `Model downloader` and convert it into OpenVINO IR model with `Model converter`.  
We'll use `googlenet-v3` model for this practice.

In [ ]:
# Linux  
!python3 $INTEL_OPENVINO_DIR/deployment_tools/open_model_zoo/tools/downloader/downloader.py --name googlenet-v3
!python3 $INTEL_OPENVINO_DIR/deployment_tools/open_model_zoo/tools/downloader/converter.py  --name googlenet-v3 --precisions FP16
!ls public/googlenet-v3/FP16 -l

In [ ]:
# Windows  
!python "%INTEL_OPENVINO_DIR%\deployment_tools\open_model_zoo\tools\downloader\downloader.py" --name googlenet-v3
!python "%INTEL_OPENVINO_DIR%\deployment_tools\open_model_zoo\tools\downloader\converter.py"  --name googlenet-v3 --precisions FP16
!dir public\googlenet-v3\FP16

----
The Python inferencing code starts from here.

### Importing required Python modules
At first, let's import required Python modules.

In [ ]:
import cv2
import numpy as np
from openvino.inference_engine import IECore

### Reading class label text file

In [ ]:
label = [ s.replace('\n', '') for s in open('synset_words.txt').readlines() ]

print(len(label), 'labels read')   # Show the number of labels
print(label[:20])                  # Show first 20 labels

### Creating an Inference Engine core object and do some preparation
- Create Inference Engine core object
- Read IR model data
- Information extraction from input and output blob (=buffers)

In [ ]:
# Create an Inference Engine core object
ie = IECore()

# Read an IR model data to memory
model = './public/googlenet-v3/FP16/googlenet-v3'
net = ie.read_network(model=model+'.xml', weights=model+'.bin')

# Obtain the name of the input and output blob, and input blob shape
input_blob_name  = list(net.input_info.keys())[0]
output_blob_name = list(net.outputs.keys())[0]
batch,channel,height,width = net.input_info[input_blob_name].tensor_desc.dims

# Show model input and output information
print(input_blob_name, batch, channel, height, width)
print(output_blob_name, net.outputs[output_blob_name].shape)

### Loading model data to the IE core object
Load the model data to the IE core object. The `CPU` is specified as the processor to use for infrencing job.  
You can change this to either one of `'GPU'`, `'MYRIAD'`, `'HETERO:FPGA,CPU'` to try another processor.  
In most case, you can switch the inferencing processor by just modifying this portion in your OpenVINO baed aplication and the rest of the code can be remain unchanged.  

**Note:** The development server of DevCloud only has CPU. You need to use DevCloud edge computing node to try other devices.

In [ ]:
exec_net = ie.load_network(network=net, device_name='CPU', num_requests=1)

### Reading and manipulate input image
Read the input image file and resize and transform it to fit it for input blob of the DL model using OpenCV.

In [ ]:
print('input blob: name="{}", N={}, C={}, H={}, W={}'.format(input_blob_name, batch, channel, height, width))
img = cv2.imread('car.png')
img = cv2.resize(img, (width,height))
img = img.transpose((2, 0, 1))
img = img.reshape((1, channel, height, width))

print(img.shape)

### Running inference  
The `infer()` API is a blocking function and the control will be kept waited until the inferencing task is completed.  
Input data can be passed with a dictionary object in `{input_blob_name:input_data}` format.

In [ ]:
res = exec_net.infer(inputs={input_blob_name: img})

print(res[output_blob_name].shape)

### Displaying the inference result  
The `googlenet-v3` model is trained with the ImageNet dataset which has 1000 classes. The model outputs probability in FP32 value for each one of 1000 classes. The output data is an array of FP32 and number of elements is 1000 (float [1000]).  
Here, we sort the probability in high to low order and display top 5 results.

In [ ]:
result=res[output_blob_name][0]
idx = np.argsort(result)[::-1]
for i in range(5):
    print(idx[i], result[idx[i]], label[idx[i]-1])

----
Now, you have learnt the most simple image classification program using OpenVINO Python API.  
OpenVINO Python API is simple yet very powerful. You can write image classification application with 20+ lines of Python code.  

Try changing input picture and see the inferencing result.

## Next => Basic of object detection program using OpenVINO - [object-detection-ssd.ipynb](./object-detection-ssd.ipynb)